<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# GR HD Equations

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

## Introduction

We write the equations of general relativistic hydrodynamics in conservative form as follows (adapted from Eqs. 41-44 of [Duez et al](https://arxiv.org/pdf/astro-ph/0503420.pdf):

\begin{array}
\ \partial_t \rho_* &+ \partial_j \left(\rho_* v^j\right) &= 0 \\
\partial_t \tilde{\tau} &+ \partial_j \left(\alpha^2 \sqrt{\gamma} T^{0j} - \rho_* v^j \right) &= s \\
\partial_t \tilde{S}_i &+ \partial_j \left(\alpha \sqrt{\gamma} T^j{}_i \right) &= \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i},
\end{array}
where we assume $T^{\mu\nu}$ is the stress-energy tensor of a perfect fluid:
$$
T^{\mu\nu} = \rho_0 h u^{\mu} u^{\nu} + P g^{\mu\nu},
$$
the $s$ source term is given in terms of ADM quantities via
$$
s = \alpha \sqrt{\gamma}\left[\left(T^{00}\beta^i\beta^j + 2 T^{0i}\beta^j + T^{ij} \right)K_{ij}
- \left(T^{00}\beta^i + T^{0i} \right)\partial_i\alpha \right],
$$

and 
\begin{align}
v^j &= \frac{u^j}{u^0} \\
\rho_* &= \alpha\sqrt{\gamma} \rho_0 u^0 \\
h &= 1 + \epsilon + \frac{P}{\rho_0}.
\end{align}

Also we will write the 4-metric in terms of the ADM 3-metric, lapse, and shift using standard equations.

Thus the full set of input variables include:
* Spacetime quantities:
    * ADM quantities $\alpha$, $\beta^i$, $\gamma_{ij}$, $K_{ij}$
* Hydrodynamical quantities:
    * Rest-mass density $\rho_0$
    * Pressure $P$
    * Internal energy $\epsilon$
    * 4-velocity $u^\mu$

For completeness, the rest of the conservative variables are given by
\begin{align}
\tilde{\tau} &= \alpha^2\sqrt{\gamma} T^{00} - \rho_* \\
\tilde{S}_i  &= \alpha \sqrt{\gamma} T^0{}_i
\end{align}

### A Note on Notation

As is standard in NRPy+, 

* Greek indices refer to four-dimensional quantities where the zeroth component indicates temporal (time) component.
* Latin indices refer to three-dimensional quantities. This is somewhat counterintuitive since Python always indexes its lists starting from 0. As a result, the zeroth component of three-dimensional quantities will necessarily indicate the first *spatial* direction.

For instance, in calculating the first term of [$T_{\rm EM}^{\mu\nu}$](#em_tensor) (specifically, ${\rm Term\ 1} = b^2 u^\mu u^\nu$), we use Greek indices:

```
T4EMUU = ixp.zerorank2(DIM=4)
for mu in range(4):
    for nu in range(4):
        # Term 1: b^2 u^{\mu} u^{\nu}
        T4EMUU[mu][nu] = smallb2*u4U[mu]*u4U[nu]\
```

When we calculate [$\beta_i = \gamma_{ij} \beta^j$](#4metric), we use Latin indices:
```
betaD = ixp.zerorank1()
for i in range(DIM):
    for j in range(DIM):
        betaD[i] += gammaDD[i][j] * betaU[j]
```

As a corollary, any expressions involving mixed Greek and Latin indices will need to offset one set of indices by one: A Latin index in a four-vector will be incremented and a Greek index in a three-vector will be decremented (however, the latter case does not occur in this tutorial module). This can be seen when we handle the second term of [$\partial_t \tilde{S}_i$](#construct_si) (or, more specifically, the second term thereof: $\frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}$):
```
# The second term: \alpha \sqrt{\\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu} / 2
for i in range(DIM):
    for mu in range(4):
        for nu in range(4):
            Stilde_rhsD[i] += alpsqrtgam * T4EMUU[mu][nu] * g4DDdD[mu][nu][i+1] / 2
```

# Import needed NRPy+ & Python modules

In [1]:
# Step 1: Import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
# import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

# Define the stress-energy tensor $T^{\mu\nu}$

Recall from above that

$$
T^{\mu\nu} = \rho_0 h u^{\mu} u^{\nu} + P g^{\mu\nu},
$$
where

$$
h = 1 + \epsilon + \frac{P}{\rho_0}
$$

In [2]:
# First define h, the enthalpy:
def compute_enthalpy(rho_b,P,epsilon):
    global h
    h = 1 + epsilon + P/rho_b

def compute_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U):
    global T4UU

    compute_enthalpy(rho_b,P,epsilon)
    # Then define g^{mu nu} in terms of the ADM quantities:
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4UU_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)

    # Finally compute T^{mu nu}
    T4UU = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            T4UU[mu][nu] = rho_b * h * u4U[mu]*u4U[nu] + P*AB4m.g4UU[mu][nu]

def compute_T4UD(gammaDD,betaU,alpha, T4UU):
    global T4UD
    # Next compute T^mu_nu = T^{mu delta} g_{delta nu}, needed for Stilde flux.
    # First we'll need g_{alpha nu} in terms of ADM quantities:
    import BSSN.ADMBSSN_tofrom_4metric as AB4m
    AB4m.g4DD_ito_BSSN_or_ADM("ADM",gammaDD,betaU,alpha)
    T4UD = ixp.zerorank2(DIM=4)
    for mu in range(4):
        for nu in range(4):
            for delta in range(4):
                T4UD[mu][nu] += T4UU[mu][delta]*AB4m.g4DD[delta][nu]

# Writing the conservative variables in terms of the primitive variables

Recall from above that the conservative variables may be written as
\begin{align}
\rho_* &= \alpha\sqrt{\gamma} \rho_0 u^0 \\
\tilde{\tau} &= \alpha^2\sqrt{\gamma} T^{00} - \rho_* \\
\tilde{S}_i  &= \alpha \sqrt{\gamma} T^0{}_i
\end{align}

$\rho_*$ has already been defined above, as has $T^{\mu\nu}$ and $T^\mu{}_\nu$ $-$ all in terms of primitive variables. Thus we have

In [3]:
def compute_vU_from_u4U__no_speed_limit(u4U):
    global vU
    # Now compute v^i = u^i/u^0:
    vU = ixp.zerorank1(DIM=3)
    for j in range(3):
        vU[j] = u4U[j+1]/u4U[0]

def compute_sqrtgammaDET(gammaDD):
    global sqrtgammaDET
    gammaUU, gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
    sqrtgammaDET = sp.sqrt(gammaDET)

def compute_rho_star(alpha, sqrtgammaDET, rho_b,u4U):
    global rho_star
    # Compute rho_star:
    rho_star = alpha*sqrtgammaDET*rho_b*u4U[0]

def compute_tau_tilde(alpha, sqrtgammaDET, T4UU,rho_star):
    global tau_tilde
    tau_tilde = alpha**2*sqrtgammaDET*T4UU[0][0] - rho_star

def compute_S_tildeD(alpha, sqrtgammaDET, T4UD):
    global S_tildeD
    S_tildeD = ixp.zerorank1(DIM=3)
    for i in range(3):
        S_tildeD[i] = alpha*sqrtgammaDET*T4UD[0][i+1]

# Define $\rho_*$ flux term for GRHD equations

Recall from above that
\begin{array}
\ \partial_t \rho_* &+ \partial_j \left(\rho_* v^j\right) = 0
\end{array}
where

$$
\rho_* = \alpha\sqrt{\gamma} \rho_0 u^0.
$$

Here we will define the $\rho_* v^j$ that goes inside the $\rho_*$ flux term:

In [4]:
### RHO_STAR FLUX
def compute_rho_star_flux(vU, rho_star):
    global rho_star_fluxU
    rho_star_fluxU = ixp.zerorank1(DIM=3)
    for j in range(3):
        rho_star_fluxU[j] = rho_star*vU[j]

# Define $\tilde{\tau}$ and $\tilde{S}_i$ flux terms for GRHD equations

Recall from above that
\begin{array}
\ \partial_t \tilde{\tau} &+ \partial_j \left(\alpha^2 \sqrt{\gamma} T^{0j} - \rho_* v^j \right) &= s \\
\partial_t \tilde{S}_i &+ \partial_j \left(\alpha \sqrt{\gamma} T^j{}_i \right) &= \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i},
\end{array}
where

Here we will define all terms that go inside the $\partial_j$'s on the left-hand side of the above equations

In [5]:
### TAUTILDE FLUX
def compute_tau_tilde_flux(alpha, sqrtgammaDET, vU,T4UU):
    tau_tilde_fluxU = ixp.zerorank1(DIM=3)
    for j in range(3):
        tau_tilde_fluxU[j] = alpha**2*sqrtgammaDET*T4UU[0][j+1] - rho_star*vU[j]

### STILDE FLUX
def compute_S_tilde_flux(gammaDD,betaU,alpha, sqrtgammaDET, T4UU):

    Stilde_fluxUD = ixp.zerorank2(DIM=3)
    for j in range(3):
        for i in range(3):
            Stilde_fluxUD[j][i] = alpha*sqrtgammaDET*T4UD[j+1][i+1]

# Define source terms on RHSs of GRHD equations

## Define $s$ source term on RHS of $\tilde{\tau}$ equation:
Recall again from above the $s$ source term on the right-hand side of the $\tilde{\tau}$ equation is given in terms of ADM quantities and the stress-energy tensor via
$$
s = \alpha \sqrt{\gamma}\left[\left(T^{00}\beta^i\beta^j + 2 T^{0i}\beta^j + T^{ij} \right)K_{ij}
- \left(T^{00}\beta^i + T^{0i} \right)\partial_i\alpha \right],
$$

In [6]:
def compute_s_source_term(KDD,betaU,alpha, sqrtgammaDET,alpha_dD, T4UU):
    global s_source_term
    s_source_term = sp.sympify(0)
    for i in range(3):
        for j in range(3):
            s_source_term += (T4UU[0][0]*betaU[i]*betaU[j] + 2*T4UU[0][i+1]*betaU[j] + T4UU[i+1][j+1])*KDD[i][j]

#     alpha_dD = ixp.declarerank1("alpha_dD",DIM=3)
    for i in range(3):
        s_source_term += -(T4UU[0][0]*betaU[i] + T4UU[0][i+1])*alpha_dD[i]

    s_source_term *= alpha*sqrtgammaDET

## Define source term on RHS of $\tilde{S}_i$ equation

Recall from above
$$
\partial_t \tilde{S}_i + \partial_j \left(\alpha \sqrt{\gamma} T^j{}_i \right) = \frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i}.
$$
Our goal here will be to compute
$$
\frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i}.
$$

### Compute $g_{\mu\nu,i}$ in terms of ADM quantities and their derivatives

To compute $g_{\mu\nu,i}$ we need to evaluate the first derivative of $g_{\mu\nu}$ in terms of ADM variables.

We are given $\gamma_{ij}$, $\alpha$, and $\beta^i$, and the 4-metric is given in terms of these quantities via
$$
g_{\mu\nu} = \begin{pmatrix} 
-\alpha^2 + \beta^k \beta_k & \beta_i \\
\beta_j & \gamma_{ij}
\end{pmatrix}.
$$

Thus 
$$
g_{\mu\nu,k} = \begin{pmatrix} 
-2 \alpha\alpha_{,i} + \beta^j_{,k} \beta_j + \beta^j \beta_{j,k} & \beta_{i,k} \\
\beta_{j,k} & \gamma_{ij,k}
\end{pmatrix}.
$$

In [7]:
def compute_g4DDdD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD):
    global g4DDdD
    # Eq. 2.121 in B&S
    betaD = ixp.zerorank1()
    for i in range(3):
        for j in range(3):
            betaD[i] += gammaDD[i][j]*betaU[j]

    # gammaDD_dD = ixp.declarerank3("gammaDD_dDD","sym12",DIM=3)
    # betaU_dD   = ixp.declarerank2("betaU_dD"   ,"nosym",DIM=3)
    betaDdD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Recall that betaD[i] = gammaDD[i][j]*betaU[j] (Eq. 2.121 in B&S)
                betaDdD[i][k] += gammaDD_dD[i][j][k]*betaU[j] + gammaDD[i][j]*betaU_dD[j][k]

    # Eq. 2.122 in B&S
    g4DDdD = ixp.zerorank3(DIM=4)
    for k in range(3):
        # Recall that g4DD[0][0] = -alpha^2 + betaU[j]*betaD[j]
        g4DDdD[0][0][k+1] += -2*alpha*alpha_dD[k]
        for j in range(3):
            g4DDdD[0][0][k+1] += betaU_dD[j][k]*betaD[j] + betaU[j]*betaDdD[j][k]

    for i in range(3):
        for k in range(3):
            # Recall that g4DD[i][0] = g4DD[0][i] = betaD[i]
            g4DDdD[i+1][0][k+1] = g4DDdD[0][i+1][k+1] = betaDdD[i][k]
    for i in range(3):
        for j in range(3):
            for k in range(3):
                # Recall that g4DD[i][j] = gammaDD[i][j]
                g4DDdD[i+1][j+1][k+1] = gammaDD_dD[i][j][k]

### Compute source term of the $\tilde{S}_i$ equation: $\frac{1}{2} \alpha\sqrt{\gamma} T^{\mu\nu} g_{\mu\nu,i}$

Now that we've computed `g4DDdD`$=g_{\mu\nu,i}$, the $\tilde{S}_i$ evolution equation source term may be quickly constructed.

In [8]:
def compute_Stilde_source_termD(alpha, sqrtgammaDET,g4DDdD, T4UU):
    global Stilde_source_termD
    Stilde_source_termD = ixp.zerorank1(DIM=3)
    for i in range(3):
        for mu in range(4):
            for nu in range(4):
                Stilde_source_termD[i] += sp.Rational(1,2)*alpha*sqrtgammaDET*T4UU[mu][nu]*g4DDdD[mu][nu][i+1]

# Conversion of $v^i$ to $u^\mu$ (Courtesy Patrick Nelson)

This conversion is a bit subtle, as truncation error on $v^i$ may result in a superluminal or ultrarelativistic velocity, which in either case a GRHD code cannot handle reliably.

As described in the [`GiRaFFE` $\tilde{S}_i$ tutorial module](Tutorial-GiRaFFE_NRPy_Ccode_library-Stilde-flux.ipynb#speed_limit), we have already devised an algorithm for computing $u^\mu$ from $v_{(n)}^i$, the Valencia 3-velocity, defined as (Eq 11 in the IllinoisGRMHD paper)

$$
v^i_{(n)} = \frac{1}{\alpha}\left(v^i + \beta^i\right).
$$

Thus we first construct the Valencia velocity from $v^i = u^i/u^0$, then call the function from `GiRaFFE` to apply when necessary the speed limit on the Lorentz factor `GAMMA_SPEED_LIMIT`, then output the limited $v^i_{(n)}$ and $u^0$.

Then from these quantities, $u^{i}$ is given by
\begin{align}
v^i_{(n)} &= \frac{1}{\alpha}\left(v^i + \beta^i\right) \\
v^i_{(n)} &= \frac{1}{\alpha}\left(\frac{u^i}{u^0} + \beta^i\right) \\
\implies \frac{u^i}{u^0} + \beta^i &= \alpha v^i_{(n)}\\
\implies u^i &= u^0 \left(\alpha v^i_{(n)} - \beta^i\right)
\end{align}

In [9]:
def u4U_in_terms_of_vU_apply_speed_limit(alpha,betaU,gammaDD, vU):
    global u4_ito_3velsU
    
    import GiRaFFE_HO.Stilde_flux as GSf
    ValenciavU = ixp.zerorank1()
    for i in range(3):
        ValenciavU[i] = (vU[i] + betaU[i])/alpha

    GSf.compute_u0_noif(gammaDD,alpha,ValenciavU)
    u4_ito_3velsU = ixp.zerorank1(DIM=4)
    u4_ito_3velsU[0] = GSf.rescaledu0
    for i in range(3):
        u4_ito_3velsU[i] = GSf.rescaledu0 * (alpha * GSf.rescaledValenciavU[i] - betaU[i])

# Declare ADM and hydrodynamical input variables, and construct GRHD equations

In [11]:
# First hydrodynamical quantities
u4U = ixp.declarerank1("u4U", DIM=4)
rho_b,P,epsilon = sp.symbols('rho_b P epsilon',real=True)

# Then ADM quantities
gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
KDD     = ixp.declarerank2("KDD"    ,"sym01",DIM=3)
betaU   = ixp.declarerank1("betaU", DIM=3)
alpha   = sp.symbols('alpha', real=True)

# First compute stress-energy tensor T4UU and T4UD:
compute_T4UU(gammaDD,betaU,alpha, rho_b,P,epsilon,u4U)
compute_T4UD(gammaDD,betaU,alpha, T4UU)

# Next sqrt(gamma)
compute_sqrtgammaDET(gammaDD)

# Compute conservative variables in terms of primitive variables
compute_rho_star( alpha, sqrtgammaDET, rho_b,u4U)
compute_tau_tilde(alpha, sqrtgammaDET, T4UU,rho_star)
compute_S_tildeD( alpha, sqrtgammaDET, T4UD)

# Then compute v^i from u^mu
compute_vU_from_u4U__no_speed_limit(u4U)

# Next compute fluxes of conservative variables
compute_rho_star_flux(vU, rho_star)
compute_tau_tilde_flux(alpha, sqrtgammaDET, vU,T4UU)
compute_S_tilde_flux(gammaDD,betaU,alpha, sqrtgammaDET, T4UU)

# Then declare derivatives & compute g4DDdD
gammaDD_dD = ixp.declarerank3("gammaDD_dD","sym01",DIM=3)
betaU_dD   = ixp.declarerank2("betaU_dD"  ,"nosym",DIM=3)
alpha_dD   = ixp.declarerank1("alpha_dD"          ,DIM=3)
compute_g4DDdD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD)

# Finally compute source terms on tau_tilde and S_tilde equations
compute_s_source_term(KDD,betaU,alpha, sqrtgammaDET,alpha_dD, T4UU)
compute_Stilde_source_termD(    alpha, sqrtgammaDET,g4DDdD,   T4UU)